# Headers

In [ ]:

import pandas as pd 
from collections import defaultdict
# from graphviz import Digraph
from graphviz import Digraph



# Class for Session Node to store time of visit for User -> POI 


In [ ]:

class SessionNode:
    def __init__(self, hourSlot):
        self.hour = hourSlot
        


# Class for User Node to store user info


In [ ]:

class UserNode:
    def __init__(self, userId):
        self.userId = userId



# Class for Point of Interest (POI) Node to store various POIs like malls, parks, restaurants, visiting places etc..


In [ ]:

class POINode:
    def __init__(self, venueId, venueTypeId=None, venueName=None, latitude=None, longitude=None):
        self.venueId = venueId
        self.venueTypeId = venueTypeId
        self.venueName = venueName
        self.location = (latitude, longitude)
        


# Class for graph that represents POIs, user info and time of visits


In [ ]:

class Graph:
    def __init__(self):
        self.userToSessionLinks = defaultdict(list) # userNode  -> [(sessionNode, weight)]
        self.sessionToPOILinks = defaultdict(list)  # sessionNode -> [(POINode, weight)]
        
        #auxiliary lookups for handling redundant data from dataset
        self.userIdToUserNodes = {}
        self.venueIdToPOINOdes = {}
    
    
    def loadGraph(self,dataset):
        data = pd.read_csv(dataset) 
        for index, row in data.head(5).iterrows():
            #print(row['userID'], row['venueID'])
            #userID,venueID,date,year,monthOfYear,dayOfWeek,dayOfMonth,dayHour,visitCount
            userNode = None
            poiNode = None
            sessionNode = None
            
            userId = row['userID']
            if userId in self.userIdToUserNodes:
                userNode = self.userIdToUserNodes[userId]
            else:
                userNode = UserNode(userId) 
                self.userIdToUserNodes[userId] = userNode
            
            venueId = row['venueID']
            if venueId in self.venueIdToPOINOdes:
                poiNode = self.venueIdToPOINOdes[venueId]
            else:
                poiNode = POINode(venueId)
                self.venueIdToPOINOdes[venueId] = poiNode
            
            hourSlot = row['dayHour']
            
            if userNode in self.userToSessionLinks:
                sessionNodeWeights = self.userToSessionLinks[userNode]
                #find hour slot in existing sessionNodes
                foundHourSlot = False
                for sNode in sessionNodeWeights:
                    #import pdb; pdb.set_trace()
                    sessionNode = sNode[0]
                    if sessionNode.hour == hourSlot:
                        sNode[1] += 1    #incrementing weight for the hour slot 
                        foundHourSlot = True
                        break
                if not foundHourSlot:
                    sessionNode = SessionNode(hourSlot)
                    self.userToSessionLinks[userNode].append([sessionNode, 1])
            
            else:
                #no userNode mapping
                sessionNode = SessionNode(hourSlot)
                self.userToSessionLinks[userNode] = [[sessionNode, 1]]
            
            
            if sessionNode in self.sessionToPOILinks:
                poiNodeWeights = self.sessionToPOILinks[sessionNode]
                foundPOINode = False
                for pNode in poiNodeWeights:
                    if pNode[0] == poiNode:
                        pNode[1] += 1
                        foundPOINode = True
                        break
                if not foundPOINode:
                    self.sessionToPOILinks[sessionNode].append([poiNode, 1])
            else:
                self.sessionToPOILinks[sessionNode] = [[poiNode, 1]]
            
    
    
    def lookupFromGraph(self, userId):
        pass
        
    

In [ ]:
graph = Graph()
graph.loadGraph("export.csv")
print(graph.userIdToUserNodes)
print ('----------------------')
print(graph.userToSessionLinks)
print ('----------------------')
print (graph.venueIdToPOINOdes)
print ('----------------------')
print (graph.sessionToPOILinks)

In [ ]:

# def visualizeGraph():
#     edges = []
#     for key in graph.userToSessionLinks:
#         userNode = key
#         sessionNodesList = graph.userToSessionLinks[key]
#         for item in sessionNodesList:
#             edges.append(tuple([userNode, item[0]]))
#     print (edges)
#     G = ig.Graph(edges, directed=False)
    

In [ ]:
## Visualizing graph using Graphviz
dot = Digraph(comment='The Round Table')

dot.node('1', 'King Arthur')
dot.node('2', 'Sir Bedevere the Wise')
dot.node('3', 'Sir Lancelot the Brave')

dot.edges(['12', '13'])
# dot.edge('B', 'L', constraint='false')
dot.render('round-table.gv', view=True) 

In [ ]:
graph = Graph()
graph.loadGraph("export.csv")
userval = None
user = graph.userIdToUserNodes
for item in user:
    userval = item
print (userval)
counter = 'a'
userNode_counter = counter
dot.node(counter, str(user))
counter = chr(ord(counter) + 1) 
sessions = graph.userToSessionLinks

dot = Digraph(comment='Sample graph for user: ' + str(userval))
edges = []

for sessionnode in sessions[user[userval]]:
    dot.node(counter , str(sessionnode[0]))
    edges.append(userNode_counter + counter)
    ##session - to - POI nodes
    sessionNode_counter = counter
    counter = chr(ord(counter) + 1)
    for poiNode in graph.sessionToPOILinks[sessionnode[0]]:
        dot.node(counter, str(poiNode[0]))
        edges.append(sessionNode_counter + counter)
        counter = chr(ord(counter) + 1)
print (edges)
dot.edges(edges)
dot.render('graph-viz.gv', view=True)